In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Github/

/content/drive/MyDrive/Github


In [3]:
!git pull https://github.com/rohit-khoiwal-30/msgmt.git

From https://github.com/rohit-khoiwal-30/msgmt
 * branch            HEAD       -> FETCH_HEAD
Already up to date.


In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("augment_train.csv")
df_test = pd.read_csv("evaluation.csv")

In [6]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Input
from keras.regularizers import l2

try:
    import tensorflow_hub as hub
except ModuleNotFoundError:
    %pip install tensorflow_hub
    import tensorflow_hub as hub
    
try:
    from keras_self_attention import SeqSelfAttention
except ModuleNotFoundError:
    %pip install -q keras-self-attention
    from keras_self_attention import SeqSelfAttention

import numpy as np


# %pip install -q transformers
# %pip install -q -U tensorflow-text
# %pip install -q tf-models-official==2.7.0

import math as m

In [7]:
def gen_random_batch(in_groups, batch_halfsize = 8):
    text_batch, reason_batch, nonreason_batch, y_hat = [], [], [], []
    all_groups = list(range(in_groups[0].shape[0]))

    group_idx = np.random.choice(all_groups, size = batch_halfsize*2)
    text_batch += [in_groups[0][c_idx] for c_idx in group_idx]
    
    b_group_idx = group_idx
    y_hat += [1]*batch_halfsize
    reason_batch += [in_groups[1][c_idx] for c_idx in b_group_idx]
    

    non_group_idx = [np.random.choice([i for i in all_groups if i!=c_idx]) for c_idx in group_idx] 
    b_group_idx = non_group_idx
    y_hat += [0]*batch_halfsize
    nonreason_batch += [in_groups[1][c_idx] for c_idx in b_group_idx]
            
    return np.stack(text_batch, 0), np.stack(reason_batch, 0), np.stack(nonreason_batch, 0), np.stack(y_hat, 0)

def siam_gen(in_groups, batch_size = 32):
    while True:
        text_stack, reason_stack, nonreason_stack, y_hatstack = gen_random_batch(in_groups, batch_size//2)
        yield [text_stack, reason_stack, nonreason_stack], y_hatstack


def identity_loss(y_true, y_pred):
    return K.mean(y_pred)

def triplet_loss(x, alpha = 0.2):
    # Triplet Loss function.
    anchor,positive,negative = x
    # distance between the anchor and the positive
    pos_dist = K.sum(K.square(anchor-positive),axis=1)
    # distance between the anchor and the negative
    neg_dist = K.sum(K.square(anchor-negative),axis=1)
    # compute loss
    basic_loss = pos_dist-neg_dist+alpha
    loss = K.maximum(basic_loss,0.0)
    return loss

In [8]:
def accuracy(y_true, y_pred):
    results = (y_pred >= 0.5).astype(int).squeeze()
    return np.mean([y_true == results])

In [9]:
huburl = "https://tfhub.dev/google/universal-sentence-encoder/4" 
fine_tuned_module_object = hub.load(huburl)

In [10]:
def encoder1(name):
    inputs = Input(shape=(), dtype=tf.string)

    shared_embedding_layer = hub.KerasLayer(fine_tuned_module_object, trainable=True, name=name)
    embedding_output= shared_embedding_layer(inputs)
    a = tfl.Dense(128, activation="linear", kernel_regularizer=l2(1e-3))(embedding_output)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)
    a = tfl.Dense(64, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)
    a = tfl.Dense(32, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    outputs = tfl.Activation('relu')(a)
    # outputs = tfl.Lambda(lambda x: K.l2_normalize(x,axis=-1))(a)
    
    model = Model(inputs, outputs)
    return model

def encoder2(name):
    inputs = Input(shape=(), dtype=tf.string)

    shared_embedding_layer = hub.KerasLayer(fine_tuned_module_object, trainable=True, name=name)
    embedding_output= shared_embedding_layer(inputs)
    a = tfl.Dense(64, activation="linear", kernel_regularizer=l2(1e-3))(embedding_output)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)
    a = tfl.Dense(32, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    outputs = tfl.Activation('relu')(a)
    # outputs = tfl.Lambda(lambda x: K.l2_normalize(x,axis=-1))(a)
    
    model = Model(inputs, outputs)
    return model

In [11]:
def get_model():
    #textEncoder
    inputText = Input(shape=(), dtype=tf.string)
    textModel = encoder1("textBertEncoder")
    text_embedd = textModel(inputText)

    #reasonEncoder
    reasonModel = encoder2("reasonBertEncoder")

    inputReason = Input(shape=(), dtype=tf.string)
    reason_embedd = reasonModel(inputReason)

    #nonReasonEncoder
    inputNotReason = Input(shape=(), dtype=tf.string)
    nonreason_embedd = reasonModel(inputNotReason)

    loss = tfl.Lambda(triplet_loss)([text_embedd, reason_embedd, nonreason_embedd])

    model = Model(inputs = [inputText, inputReason, inputNotReason], outputs = [loss], name="bertModel")
    return model, textModel, reasonModel

In [12]:
model, textModel, reasonModel = get_model()

In [13]:
model.summary()

Model: "bertModel"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 input_5 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 model (Functional)             (None, 32)           256874720   ['input_1[0][0]']                
                                                                                          

In [14]:
model.compile(optimizer="adam", loss = identity_loss)

In [15]:
eval_df = df_test[df_test['label'] == 1]

In [16]:
loss_history = model.fit(siam_gen([df["texts"], df["reason"]], 64), 
                         validation_data = siam_gen([eval_df["text"].values, eval_df["reason"].values], 64),
                         steps_per_epoch=50, validation_steps=10, epochs = 100, 
                         verbose = True, use_multiprocessing=True)

Epoch 1/100
50/50 [==============================] - 28s 282ms/step - loss: 2.9999 - val_loss: 0.6920
Epoch 2/100
50/50 [==============================] - 12s 236ms/step - loss: 1.9503 - val_loss: 0.7358
Epoch 3/100
50/50 [==============================] - 14s 273ms/step - loss: 1.5585 - val_loss: 0.7578
Epoch 4/100
50/50 [==============================] - 13s 254ms/step - loss: 1.4395 - val_loss: 0.9416
Epoch 5/100
50/50 [==============================] - 12s 247ms/step - loss: 1.2036 - val_loss: 1.0293
Epoch 6/100
50/50 [==============================] - 14s 276ms/step - loss: 1.1055 - val_loss: 1.0692
Epoch 7/100
50/50 [==============================] - 12s 249ms/step - loss: 1.0404 - val_loss: 1.0766
Epoch 8/100
50/50 [==============================] - 12s 248ms/step - loss: 0.9227 - val_loss: 1.0125
Epoch 9/100
50/50 [==============================] - 12s 250ms/step - loss: 0.9335 - val_loss: 0.8822
Epoch 10/100
50/50 [==============================] - 12s 248ms/step - loss: 0.808

In [17]:
left_input = tf.keras.Input(shape=(), dtype=tf.string)
right_input = tf.keras.Input(shape=(), dtype=tf.string)

textEmbedding = textModel(left_input)
reasonEmbedding = reasonModel(right_input)

cosine_similiarity=tf.keras.layers.Dot(axes=-1,normalize=True)([textEmbedding, reasonEmbedding])


classifier_model = Model(inputs = [left_input, right_input], outputs = [cosine_similiarity], name="classifier_model")

In [18]:
classifier_model.summary()

Model: "classifier_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 input_7 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 model (Functional)             (None, 32)           256874720   ['input_6[0][0]']                
                                                                                                  
 model_1 (Functional)           (None, 32)           256833120   ['input_7[0][0]']                
                                                                                   

In [19]:
y_preds = classifier_model.predict([df['texts'], df['reason']])
accuracy(df["label"], y_preds)

0.7417977802170996

In [20]:
from sklearn.metrics import brier_score_loss as brier_loss
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss

In [21]:
y_hat = (y_preds >= 0.5).astype(int)

In [22]:
print("Brier Loss                       : ", brier_loss(df['label'], y_preds))
print("Precision Score                  : ", precision_score(df['label'], y_hat))
print("Recall Score                     : ", recall_score(df['label'], y_hat))
print("F1 Score                         : ", f1_score(df['label'], y_hat))
# print("Roc AUC Score                    : ", roc_auc_score(df['label'], y_preds))
# print("BCE Loss                         : ", log_loss(df['label'], y_preds))

Brier Loss                       :  0.1991645213755615
Precision Score                  :  1.0
Recall Score                     :  0.7417977802170996
F1 Score                         :  0.8517610811567817


In [23]:
max(y_preds), min(y_preds)

(array([0.91730636], dtype=float32), array([0.], dtype=float32))

In [24]:
y_preds = classifier_model.predict([df_test['text'], df_test['reason']])
accuracy(df_test["label"], y_preds)

0.5538888888888889

In [25]:
max(y_preds), min(y_preds)

(array([0.9213097], dtype=float32), array([0.], dtype=float32))

In [26]:
y_hat = (y_preds >= 0.5).astype(int)

In [27]:
print("Brier Loss                       : ", brier_loss(df_test['label'], y_preds))
print("Precision Score                  : ", precision_score(df_test['label'], y_hat))
print("Recall Score                     : ", recall_score(df_test['label'], y_hat))
print("F1 Score                         : ", f1_score(df_test['label'], y_hat))
print("Roc AUC Score                    : ", roc_auc_score(df_test['label'], y_preds))
print("BCE Loss                         : ", log_loss(df_test['label'], y_preds))

Brier Loss                       :  0.2478094855944127
Precision Score                  :  0.40835140997830804
Recall Score                     :  0.7527490836387871
F1 Score                         :  0.5294738075706082
Roc AUC Score                    :  0.655696281491767
BCE Loss                         :  0.7203542785120921


In [ ]:
classifier_model.save("models/useTripletLoss/")